# `scrontab` Guide
`scrontab` lets a user submit recurring jobs to the Slurm cluster, similar to, as you can tell from the name, a cron job in Linux. One major difference between a cron job and a `scrontab` recurring job is that a cron job is expected to run at around the scheduled time, but the `scrontab` job only starts queuing at the scheduled time. Only after the current run has finished will the next run be queued. So if you want your `scrontab` job to run on time, you might need to consider dedicating some resources to it.  

reference: [scrontab - manpage](/doc/scrontab.html)

In this guide, it is better and easier to edit the `scrontab` jobs interactively. You can open a terminal alongside this notebook to try out the examples. 

### Enabling scrontab
To enable `scrontab`, you must set `ScronParameters` in `slurm.conf` to "enable"

In [ ]:
scontrol show config | grep -i scron

Then you can test if it is enabled by trying to list the `scrontab` jobs of the current user.

In [ ]:
scrontab -l

## Your first job to scrontab
As a simple example, we are going to run the following job script every 5 min.

In [ ]:
cat scrontab-example.sh

Run this command to start editing the `scrontab` job.  
```bash
scrontab -e
```  
Add the following line and save it
```
#SCRON -t 00:02:00
*/5 * * * * ~/tutorials/scrontab-example.sh
```  

Job options are specified by #SCRON directive, similar to #SBATCH. Each line of #SCRON option affect the single following job entry only. 


You can now run squeue to check if the job has been submitted. You can see the job is pending with the reason "BeginTime". Unlike a normal cron job, which is expected to start running at around the planned time, a scrontab job only start queuing when the planned time is reached. This job is scheduled to run every 5 min, with a timelimit of 2 min, specified by the #SCRON directive

In [ ]:
squeue -la --start

Now you can run this squeue command in the terminal to monitor the job
```
squeue -la -i 5
```
Once the job finished, you can observe that the job actually return to PENDING state with the same jobid, waiting for the next cycle. If you look at the job detail, by `scontrol show job`, you likely see the stdout file stay the same as well, which means the job output will be overwritten by the latest run. If you need to keep the log of every run, you could refer to `exec` line in the script, that redirect and split the io to create another script file.

## Skipping
To skip a run, you should requeue the job instead of cancelling it.  
```
scontrol requeue <jobid>
```

## Removing/disable the job
If you `scancel` the job, all future will be canceled, and the job will be disable. You can try the following:
```
scancel <jobid>
```  
Then you can list scrontab jobs (`scrontab -l`) and see the cron job has been commented out and disabled. 
```
#DISABLED: #SCRON -t 00:02:00
#DISABLED: */5 * * * * ~/tutorials/scrontab-example.sh
```

## Unfinished jobs
A new iteration will only start queuing after the previous run in completed. Let's create another job to demonstrat it. 
```
#SCRON -t 00:03:00
*/2 * * * * ~/tutorials/scrontab-example.sh
```
This is a 3 minutes job scheduled to run every 2 miniutes. However, you will find the job only runs every 4 minutes. 

## Clean UP

In [ ]:
# clean up all scrontab jobs
scrontab <<EOF
EOF